## Model Training: Convolutional Neural Network (CNN)

The purpose of this notebook is to train several Convolutional Neural Network (CNN) models for earthquake detection and property prediction. Rather than using the raw seismic signals as training data, we use images of the earthquake waveforms and spectrograms (visual representations of the spectrum of frequencies in the signal) that were created using the `Data_Preprocessing` module and the `data_preprocessing.ipynb` notebook in this repository. Using these images, we train and evaluate the following models:

1. Classification CNN model to predict whether a signal is 'earthquake' or 'noise'
2. Regression CNN model to predict earthquake magnitude
3. Regression CNN model to predict earthquake p-wave arrival time
4. Regression CNN model to predict earthquake s-wave arrival time

Model architectures used in this notebook are located in the `architectures` module of this repo. Training utilities including train/val/test split data preparation, evaluation functions, and plotting functions are located in the `training_utils` module of this repo.

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Environment setup

In [ ]:
import gc
import sys
import time

sys.path.append('/kaggle/input/pyscripts')

import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, roc_curve, auc
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

import architectures
import training_utils

plt.ioff() # Turn off matplotlib interactive mode. Prevents  memory leakage.

### Load datasets

These datasets were pre-processed from their raw STEAD format using the `DataPreprocessing` module and `data_preprocessing` notebook contained within this repo.

In [ ]:
# Load extracted raw signals
raw_signals = np.load('/kaggle/input/20kdata/extracted_raw_signals_subsample_20000.npy')

# Load created waveform images
waveform_imgs = np.load('/kaggle/input/20kdata/created_waveform_images_subsample_20000.npy')

# Load created spectrogram images
spectrogram_imgs = np.load('/kaggle/input/20kdata/created_spectrogram_images_subsample_20000-002.npy')

# Load metadata
metadata = pd.read_feather('/kaggle/input/20kdata/extracted_metadata_subsample_20000.feather')
metadata = metadata.reset_index()

#### Inspect image datasets and metadata to confirm they look as expected

In [ ]:
spectrogram_imgs.shape

In [ ]:
waveform_imgs.shape

In [ ]:
metadata

In [ ]:
metadata.columns

### Create subsets of earthquake-only data (no noise samples)
This is done in preparation for later training of regression models to predict earthquake parameters.

In [ ]:
earthquakes_indices = np.array(list(metadata[metadata['trace_category']=='earthquake_local'].index))
raw_signals_earthquakes_only = raw_signals[earthquakes_indices]
waveform_imgs_earthquakes_only = waveform_imgs[earthquakes_indices]
spectrogram_imgs_earthquakes_only = spectrogram_imgs[earthquakes_indices]
metadata_earthquakes_only = metadata[metadata['trace_category']=='earthquake_local']

### Create labels for the classification model

In [ ]:
metadata['label'] = [1 if label=='earthquake_local' else 0 for label in metadata['trace_category']]
classifier_labels = metadata['label'].values

In [ ]:
classifier_labels

### Parameter setup

In [ ]:
spectrogram_kwargs = {'image_size' : (150,100)}
waveform_kwargs = {'image_size' : (300,100)}

## Model training & evaluation

### 1. Classification CNN model training & evaluation to predict whether signals are 'earthquake' or 'noise'

In [ ]:
# Set up model callbacks
epochs = 20
callbacks_c = architectures.callbacks_setup(model_tag='cnn_classification', epochs=epochs)

# Fit model
train_dataset_c, val_dataset_c, test_dataset_c = training_utils.prepare_datasets(imgs=spectrogram_imgs, labels=classifier_labels, preproc_func=architectures.image_preprocessing, preproc_func_kwargs=spectrogram_kwargs, batch_size=32)
classifier_model = architectures.build_compile_classification_cnn(learning_rate=1e-5, loss='binary_crossentropy', metrics=['accuracy'])
classifier_history = classifier_model.fit(train_dataset_c, epochs=epochs, callbacks=callbacks_c, validation_data=val_dataset_c)

# Evaluate model and plot results
training_utils.evaluate_classification_model(classifier_model, train_dataset_c, test_dataset_c)
training_utils.plot_training_history(classifier_history)
classifier_model.summary()

### 2. Regression CNN model training & evaluation to predict earthquake magnitude

In [ ]:
epochs = 25
callbacks_mag = architectures.callbacks_setup(model_tag='cnn_regression_magnitude', epochs=epochs)

labels = np.array(metadata_earthquakes_only['source_magnitude'])
train_dataset_mag, val_dataset_mag, test_dataset_mag = training_utils.prepare_datasets(imgs=spectrogram_imgs_earthquakes_only, labels=labels, preproc_func=architectures.image_preprocessing, preproc_func_kwargs=spectrogram_kwargs, use_scaler=False, batch_size=64)
model_magnitude = architectures.build_compile_regression_cnn(learning_rate=1e-4, loss='mse', metrics=['mae'])
model_magnitude_history = model_magnitude.fit(train_dataset_mag, epochs=epochs, callbacks=callbacks_mag, validation_data=val_dataset_mag)

training_utils.evaluate_regression_model(model_magnitude, train_dataset_mag, test_dataset_mag, 'Source Magnitude', scaler=None)
training_utils.plot_training_history(model_magnitude_history)
model_magnitude.summary()

### 3. Regression CNN model training & evaluation to predict earthquake P-wave arrival time

In [ ]:
# Set up model callbacks
epochs = 50
callbacks_pwave = architectures.callbacks_setup(model_tag='cnn_regression_pwave', epochs=epochs)

# Fit model
labels = np.array(metadata_earthquakes_only['p_arrival_sample'])
train_dataset_pwave, val_dataset_pwave, test_dataset_pwave = training_utils.prepare_datasets(imgs=waveform_imgs_earthquakes_only, labels=labels, preproc_func=architectures.image_preprocessing, preproc_func_kwargs=waveform_kwargs, use_scaler=False, batch_size=32)
model_pwave = architectures.build_compile_regression_cnn(learning_rate=1e-5, loss='mse', metrics=['mae'])
model_pwave_history = model_pwave.fit(train_dataset_pwave, epochs=epochs, callbacks=callbacks_pwave, validation_data=val_dataset_pwave)

# Evaluate model
training_utils.evaluate_regression_model(model_pwave, train_dataset_pwave, test_dataset_pwave, 'P-Wave Arrival', scaler=None)
training_utils.plot_training_history(model_pwave_history)
model_pwave.summary()

### 4. Regression CNN model training & evaluation to predict earthquake S-wave arrival time

In [ ]:
# Set up model callbacks
epochs = 50
callbacks_swave = architectures.callbacks_setup(model_tag='cnn_regression_swave', epochs=epochs)

# Fit model
labels = np.array(metadata_earthquakes_only['s_arrival_sample'])
train_dataset_swave, val_dataset_swave, test_dataset_swave = training_utils.prepare_datasets(imgs=waveform_imgs_earthquakes_only, labels=labels, preproc_func=architectures.image_preprocessing, preproc_func_kwargs=waveform_kwargs, use_scaler=False, batch_size=32)
model_swave = architectures.build_compile_regression_cnn(learning_rate=1e-3, loss='mse', metrics=['mae'])
model_swave_history = model_swave.fit(train_dataset_swave, epochs=epochs, callbacks=callbacks_swave, validation_data=val_dataset_swave)

# Evaluate model
training_utils.evaluate_regression_model(model_swave, train_dataset_swave, test_dataset_swave, 'S-Wave Arrival', scaler=None)
training_utils.plot_training_history(model_swave_history)
model_swave.summary()